In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install transformers
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 32.0 MB/s 
     |████████████████████████████████| 6.6 MB 52.1 MB/s 
     |████████████████████████████████| 596 kB 61.7 MB/s 
     |████████████████████████████████| 101 kB 14.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, tqdm_notebook
from keras import optimizers
from transformers import AdamW
from transformers import ElectraForTokenClassification, ElectraTokenizerFast
import torch
import numpy as np
from transformers import BertForTokenClassification
import pandas as pd

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/2022_lesik_workspace/말뭉치/dataset.tsv', sep = '\t', keep_default_na=False)
df.head()

,text,label
0,[횡설수설/권순활]北 ‘외화벌이’ 뜯어먹기,O O O O O O PS_NAME PS_NAME O LCP_COUNTRY O O ...
1,필리핀 국민의 약 10%인 800만 명은 세계 곳곳에서 건설노동자 가정부 유모 등으...,LCP_COUNTRY O O QT_PERCENTAGE QT_PERCENTAGE QT...
2,본국의 가족에게 보내는 송금 총액은 매년 100억 달러를 넘어 필리핀 경제를 지탱하...,O O CV_RELATION O O O O O O O O QT_PRICE QT_PR...
3,멕시코 파키스탄 방글라데시 베트남 역시 해외파견 근로자의 송금이 한몫을 한다.,LCP_COUNTRY LCP_COUNTRY LCP_COUNTRY LCP_COUNTR...
4,"한국도 과거 개발연대 시절 서독에 나간 광원과 간호사, 베트남과 중동에 진출한 근로...",LCP_COUNTRY O O O O O LCP_COUNTRY O O CV_OCCUP...


데이터 레이블 리스트 형성

In [ ]:


print(df['label'].values)


['O O O O O O PS_NAME PS_NAME O LCP_COUNTRY O O O O O O O'
 'LCP_COUNTRY O O QT_PERCENTAGE QT_PERCENTAGE QT_PERCENTAGE O QT_MAN_COUNT QT_MAN_COUNT QT_MAN_COUNT O O O O O CV_OCCUPATION CV_OCCUPATION CV_OCCUPATION CV_OCCUPATION CV_OCCUPATION CV_OCCUPATION O O O O O O O'
 'O O CV_RELATION O O O O O O O O QT_PRICE QT_PRICE QT_PRICE O O LCP_COUNTRY O O O O O CV_RELATION O O O O O O'
 ... 'O CV_OCCUPATION O O'
 'O O OGG_POLITICS OGG_POLITICS OGG_POLITICS O O O O'
 'DT_WEEK DT_WEEK O O O O O O']


In [ ]:
# Split labels based on whitespace and turn them into a list
unique_labels = set()
for label in df['label'].values.tolist():
  for i in label.split():
    if i not in unique_labels:
      unique_labels.add(i)


print(unique_labels)

# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

{'LCP_CITY', 'AFA_ART_CRAFT', 'QT_ALBUM', 'CV_POSITION', 'TM_SPORTS', 'PT_FLOWER', 'AF_MUSICAL_INSTRUMENT', 'AF_TRANSPORT', 'PS_NAME', 'EV_SPORTS', 'QT_PERCENTAGE', 'QT_SPEED', 'TR_HUMANITIES', 'QT_SIZE', 'CV_SPORTS', 'OGG_OTHERS', 'DT_YEAR', 'MT_METAL', 'OGG_SPORTS', 'OGG_LAW', 'CV_CULTURE', 'TI_MINUTE', 'DT_DURATION', 'TMIG_GENRE', 'AM_FISH', 'CV_SPORTS_POSITION', 'FD_SOCIAL_SCIENCE', 'PT_TYPE', 'QT_AGE', 'AM_TYPE', 'QT_OTHERS', 'CV_DRINK', 'TMI_PROJECT', 'OGG_MEDICINE', 'TMI_HW', 'CV_LAW', 'PT_OTHERS', 'DT_OTHERS', 'LCG_MOUNTAIN', 'OGG_EDUCATION', 'QT_WEIGHT', 'LCG_OCEAN', 'QT_MAN_COUNT', 'AM_MAMMALIA', 'QT_PHONE', 'OGG_MEDIA', 'CV_POLICY', 'PT_PART', 'CV_CLOTHING', 'OGG_MILITARY', 'OGG_RELIGION', 'LCG_RIVER', 'MT_ROCK', 'DT_MONTH', 'TI_SECOND', 'DT_WEEK', 'TI_DURATION', 'LCG_CONTINENT', 'OGG_SCIENCE', 'QT_VOLUME', 'MT_CHEMICAL', 'TM_COLOR', 'LCP_PROVINCE', 'CV_LANGUAGE', 'TI_HOUR', 'OGG_ECONOMY', 'TR_MEDICINE', 'TMI_MODEL', 'EV_OTHERS', 'EV_ACTIVITY', 'FD_SCIENCE', 'O', 'TR_SOCIAL_

토큰화

In [ ]:
# Let's take a look at how can we preprocess the text - Take first example
texts = df['text'].values.tolist()


In [ ]:
model = ElectraForTokenClassification.from_pretrained("monologg/koelectra-base-v2-discriminator")
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v2-discriminator")
#text_tokenized = tokenizer(example, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

#print(text_tokenized)
#print(tokenizer.decode(text_tokenized.input_ids[0]))
#print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: ['classifier

토큰화 후 레이블 조정

In [ ]:
'''
#print(text_tokenized)
word_ids = text_tokenized.word_ids()
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))
print(word_ids)
print()
'''

'\n#print(text_tokenized)\nword_ids = text_tokenized.word_ids()\nprint(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))\nprint(word_ids)\nprint()\n'

동일한 토큰에 속하는 모든 하위 단어 간에 동일한 레이블을 제공합니다. 없는 모든 토큰은 word_ids'-1'으로 레이블이 지정됩니다.

In [ ]:
#label = labels[36]

#If we set label_all_tokens to True.....
label_all_tokens = True

#new_label = align_label_example(text_tokenized, label)
#print(new_label)
#print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))

데이터세트 클래스

In [ ]:
def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-1)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-1)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -1)
            except:
                label_ids.append(-1)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):
        lb = [i.split() for i in df['label'].values.tolist()]
        txt = df['text'].values.tolist()
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [ ]:
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

print(df_train[0:10])

                                                     text  \
140174                                  근데 밥은 한 끼도 안 먹었고?   
18801                           “철없던 시절에는 예솔이라는 이름이 싫었어요.   
42867                                장학금의 ‘나비효과’… 2년새 36억   
137916                       근데 이 굿와이프의 재밌는 점은 구성이 항상 다르고   
91340                 광복 뒤 맞이한 5월 첫째 일요일은 1946년 5월 5일이었다.   
143150                      어떤 사람이 옛한글자 팔백 몇 자를 만들 수 있대요.   
117851                      중국이 유럽연합(EU)과 ‘태양광 전쟁’에 돌입했다.   
59641   그러나 승객 91명이 골절과 찰과상 등 비교적 가벼운 상처를 입었다고 AFP통신이 ...   
25110   IS 공격을 피해 국경을 넘어 터키로 피신한 쿠르드족 난민도 18만6000여 명에 ...   
166771            불러서 이제 가장 먹고싶은 게 그 한국음식 된장찌개 뭐 이런 거드라구요   

                                                    label  
140174          O CV_FOOD O QT_COUNT QT_COUNT O O O O O O  
18801   O O O O O O O PS_NAME PS_NAME PS_NAME O O O O ...  
42867   O O O TR_SOCIAL_SCIENCE TR_SOCIAL_SCIENCE O DT...  
137916  O O AFA_VIDEO AFA_VIDEO AFA_VIDEO O O O O O O ...  
91340   O O O O DT_MONTH DT_

모델 빌딩

In [ ]:
class ElectraModel(torch.nn.Module):

    def __init__(self):

        super(ElectraModel, self).__init__()
        self.electra = ElectraForTokenClassification.from_pretrained("monologg/koelectra-base-v2-discriminator", num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.electra(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

훈련 루프

In [ ]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=1, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    #device = torch.device("cuda:0")

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label[0].to(device)
            mask = train_data['attention_mask'][0].to(device)
            input_id = train_data['input_ids'][0].to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)
            logits_clean = logits[0][train_label != -100]
            label_clean = train_label[train_label != -100]

            predictions = logits_clean.argmax(dim=1)

            acc = (predictions == label_clean).float().mean()
            total_acc_train += acc
            total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label[0].to(device)
            mask = val_data['attention_mask'][0].to(device)

            input_id = val_data['input_ids'][0].to(device)

            loss, logits = model(input_id, mask, val_label)

            logits_clean = logits[0][val_label != -100]
            label_clean = val_label[val_label != -100]

            predictions = logits_clean.argmax(dim=1)          

            acc = (predictions == label_clean).float().mean()
            total_acc_val += acc
            total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 1e-2
EPOCHS = 5

model = ElectraModel()
train_loop(model, df_train, df_val)

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: ['classifier

In [ ]:
path = "/content/gdrive/MyDrive/2022_lesik_workspace/model/KoELECTRA.pt"
# 저장하기
torch.save(model.state_dict(), path)

테스트 데이터에 대한 모델 평가

In [ ]:
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    for test_data, test_label in test_dataloader:

        test_label = test_label[0].to(device)
        mask = test_data['attention_mask'][0].to(device)
        input_id = test_data['input_ids'][0].to(device)
          
        loss, logits = model(input_id, mask, test_label.long())

        logits_clean = logits[0][test_label != -1]
        label_clean = test_label[test_label != -1]

        predictions = logits_clean.argmax(dim=1)
              
        acc = (predictions == label_clean).float().mean()
        total_acc_test += acc

    val_accuracy = total_acc_test / len(df_test)
    print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')


evaluate(model, df_test)

In [ ]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-1)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-1)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'][0].unsqueeze(0).to(device)

    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -1]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)


파를 송송 썰고, 양파를 넣어주세요.
['B-eve', 'B-art', 'B-art', 'B-art', 'B-art', 'B-eve', 'B-art', 'B-eve', 'B-art', 'B-art', 'B-art', 'B-art', 'B-art', 'B-eve']


In [ ]:
model = ElectraForTokenClassification.from_pretrained("monologg/koelectra-base-v2-discriminator")
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v2-discriminator")
word_ids = text_tokenized.word_ids()

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: ['classifier

In [ ]:
example = '2009년 7월 FC서울을 떠나 잉글랜드 프리미어리그 볼턴 원더러스로 이적한 이청용은 크리스탈 팰리스와 독일 분데스리가2 VfL 보훔을 거쳐 지난 3월 K리그로 컴백했다. 행선지는 서울이 아닌 울산이었다'
text_tokenized = tokenizer(example, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
evaluate_one_text(model, example)
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))
print(word_ids)

2009년 7월 FC서울을 떠나 잉글랜드 프리미어리그 볼턴 원더러스로 이적한 이청용은 크리스탈 팰리스와 독일 분데스리가2 VfL 보훔을 거쳐 지난 3월 K리그로 컴백했다. 행선지는 서울이 아닌 울산이었다
['B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve', 'B-eve']
['[CLS]', '2009', '##년', '7', '##월', 'FC', '##서울', '##을', '떠나', '잉글랜드', '프리미어리그', '볼턴', '원더', '##러스', '##로', '이적', '##한', '이청용', '##은', '크리스탈', '팰리스', '##와', '독일', '분데스리가', '##2', 'V', '##f', '##L', '보', '##훔', '##을', '거쳐', '지난', '3', '##월', 'K', '##리그', '##로', '컴백', '##했', '##다', '.', '행', '##선지', '##는', '서울', '##이', '아닌', '울산', '##이', '##었', '##다', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '